# Navigation Project Report
Andrey Naumov

# Agent
The agent is a deep neural network (DNN) with two hidden layers, each with 20 nodes.  All layers are fully connected.  All nodes compute linear combinations (with biases) of their inputs followed by an RELU activation.

The agent's input is the 37 dimensional state vector, and its output is a set of 4 nodes.  Each output node reports the Q value of one of the four actions given the input state, $Q(s,a)$.  The action with the maximum Q value is the one that the agent determines to be the best.

# Learning
The agent learns via a variant of Q learning called Deep Q Learning.  Every time the agent chooses an action for a new state of the environment, it computes:

\begin{equation}
Q(s(n),a(n)) \leftarrow r(s(n),a(n)) + \gamma * max_{a'}Q(s(n+1),a') \ \ \ \ \ (1)
\end{equation}

where n is the timestep, s is the state, a is the action, r is the reward, and Q is the Q value -- the network's estimate of the quality of taking action a(n) when observing state s(n).  $\gamma$ determines the lookahead time over which (discounted) rewards are summed and included in the Q estimate.  Roughly, the lookahead time is $t_0$ where $exp(-1/t_0) = \gamma$, or $t_0 = -1/ln( \gamma )$.  In the code $\gamma$ is specified as the hyperparameter `GAMMA=0.99`.

\[Note $\leftarrow$ indicates assignment; an update of $Q(s(n), a(n))$.\]

Since $Q(s(n), a(n))$ is computed via function approximation (rather than tabulation), we can't perform the assignment (1) directly.  Instead, an optimizer (ex., SGD or Adam as in this project) varies the parameters of the function approximator (the weights of the DNN) in such a way as to minimize the difference between the two sides of the equation (1).

Specifically, SGD (Stochastic Gradient Descent) calculates the gradient of the squared loss, $L$ of a *single data point*:

\begin{equation}
    L = ( r(s(n),a(n)) + \gamma * max_{a'}Q(s(n+1),a') - Q(s(n),a(n)) )^2
\end{equation}

with respect to $w$, the weights of the DNN.  The it adjusts each weight slightly in the direction that would reduce the L:

\begin{equation}
 \Delta w_i \propto \frac{\partial L}{\partial w_i} 
\end{equation}

The constant of proportionality is a hyperparameter called the learning rate (`LR=.0005`, in the code).

Gradient estimates based on a single sample can be noisy and slow down learning, so in practice we collect several samples -- a batch -- and calculate an update based on all of them.  The number of samples used for a single update is the called the batch size (`BATCH_SIZE=64` in the code).

### Adam
The Adam optimizer automatically adapts the learning rate to each parameter and reduces the learning rate as learning progresses.  This method is believed to be more efficient than SGD in many setttings.


## Exploration vs. Exploitation
To be sure it samples (or improve its chances of sampling) the whole state-action space, the agent will randomly choose actions (exploration), but to improve its chances of increasing its reward (collecting yellow bananas) it will bias those choices towards the action it thinks is best (exploitation).  Specifically, it uses $\epsilon$-greedy action selection: 

\begin{equation}
  a(n)=\begin{cases}
    argmax_{a'}Q(s(n),a'), & \text{with probability 1-$\epsilon$}.\\
    uniformly\ random\  action, & \text{otherwise}.
  \end{cases}
\end{equation}

The tradeoff between exploration and exploitation is governed by $\epsilon$, a tunable hyperparameter of the learning process.  In the code (in function `dqn()`) $\epsilon$ is decayed from `eps_start=1.0` to `eps_end=0.01` at an exponential rate of `eps_decay=0.955`.

## Forgetting
The $Q(s,a)$ update equation (1) is incremental -- as is the optimizer which updates the DNN weights -- which means that recent data was used to make the most recent network weight updates.  These updates might undo older changes made based on older data.  This can result in the network being biased towards more recent data.  

Additionally, the optimization algorithms (ex., SGD and Adam) assume that data samples are independent (uncorrelated).  It is often the case that consecutive states in dynamical systems are highly correlated (i.e., states are autocorrelated).  If multiple, consecutive states are about the same, then the associated (multiple, consecutive) weight updates will be about the same.

\[Intuitively, imagine your state includes the position and velocity of your agent.  The position of your agent (in any reasonable physics simulation) now is *about the same* as it was a moment ago.  Additionally, your agent's velocity now is *about the same* as it was a moment ago.  Your agent's position and velocity are each autocorrelated.\]

To keep from "forgetting" old data and to help break the correlations of sequential data, we store all (s,a,r) samples in a buffer then randomly select samples from this buffer to pass to the update equation (1).  This is call experience replay.

Experience reply introduces the buffer size (`BUFFER_SIZE=100000`) as a hyperparameter.

## Numerical Stability
The incremental optimizers' weight update equations are derived from supervised learning algorithms that attempt to minimize the difference between a signal (x$\beta$) and a response (y), like:

\begin{equation}
\epsilon = (y - x\beta)
\end{equation}

An optimizer like SGD could be used on sequential data to find the value of the parameter $\beta$ and it can be shown that the optimization converges.

To analogize this to Q-learning, set

\begin{equation}
y = r(s(n),a(n)) + \gamma * max_{a'}Q(s(n+1),a'; w)
\end{equation}

\begin{equation}
x\beta = Q(s(n),a(n); w)
\end{equation}

\begin{equation}
\beta = w
\end{equation}

The response, y, is the reward plus the best next Q value, the signal is the current estimate of the Q value, and the parameter to optimize is w (a DNN weight vector).

The problem with this analogy is that in Q-learning, the reponse depends on the parameter.  This wasn't the case in the supervised learning problem.  When the optimizer updates w it is changing both the signal and the response.  This can cause numerical instability in w (ex., w can grow unbounded).

One way to fix this is to use a copy of the DNN for the response (aka, target) Q value estimate.  This copy DNN -- called the target network -- keeps its weights fixed for many updates.  For that period of time the problem is more like a supervised learning problem with a fixed response and the incremental optmizer works well.

Rather than keeping the target network fixed for many updates, we can instead allow its weights to change *very* slowly by setting them to a long-term EWMA of the original Q network's weights.  This is called a soft update and introduces another hyperparameter -- the long-term averaging coefficient, `TAU=.001`.

## Result

The agent's score during learning is shown in Fig. 1.

The agent needed approximately 625 episodes to solve the problem.  Here "solved" is defined as producing a 100-episode run with an average score of at least 13.

The weights learned in the session 

Figure 1:![](result.png)

## Improvements

A broader exploration of network architectures and hyperparameter values could yield an agent that learns more quickly.  A deeper network could give greater precision.  A systematic search of hyperparameter space (ex., with a black-box optimizer), could yield better hyperparameters.  Such a search could take a long time, however, since each network training "session" time is measured in hours.